In [1]:
import setup_jwlab
import pandas as pd
import numpy as np

from jwlab.ml_prep import load_ml_df, y_to_binary
from sklearn.model_selection import cross_val_score

In [2]:
df = load_ml_df("~/projects/def-jwerker/kjslakov/data/ml_df_sktime.pkl")

In [3]:
y = y_to_binary(df.label.values.flatten())
df = df.drop(columns=["label"], axis=1)

In [7]:
from scipy.stats import entropy
funcs = [np.mean, np.min, np.max, np.var]
dfs_applied = [df.applymap(f) for f in funcs]
df_concat = pd.concat(dfs_applied, axis=1)

In [8]:
df_concat.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,9.619437,26.061406,21.843268,18.845386,-47.679006,5.239017,17.645599,20.143882,8.135744,-13.110967,...,67.099200,99.904397,69.053893,95.097644,483.463507,147.589687,195.579365,338.496411,465.947544,739.580266
1,6.853466,0.348400,12.250030,23.596510,-1.851088,16.716675,21.320742,7.950176,19.992376,9.125198,...,166.442776,149.223628,170.715839,266.217102,110.324800,144.027112,108.476078,293.564479,126.066217,157.393703
2,-18.879231,-4.448306,1.029752,4.498039,-19.878193,-1.135712,7.389829,-7.171245,6.062628,-22.587683,...,154.371511,171.381713,122.252130,221.909575,354.062099,187.940980,114.251767,228.500961,147.114253,132.507189
3,13.288054,-14.234266,-16.283855,-17.121040,-11.281634,-9.797145,-23.788907,5.723077,-19.165705,3.194474,...,269.153942,491.218658,135.598056,273.135150,120.663809,152.397632,96.231220,272.975938,97.311177,200.758434
4,20.522480,-3.583972,2.115232,13.712213,10.745049,12.048008,15.790093,5.775336,9.180519,16.967114,...,183.329085,337.916649,166.177765,255.189500,233.660702,249.016941,159.801987,386.286895,162.845653,164.711556


In [10]:
df_normalized = (df_concat - df_concat.mean()) / df_concat.std()
df_normalized.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.176785,1.170234,1.116690,0.668880,-0.988135,0.160344,0.757075,0.786278,0.313916,-0.124054,...,-0.043456,-0.042991,-0.039690,-0.054516,-0.061582,-0.040399,-0.038211,-0.060736,-0.035664,-0.030308
1,0.101716,-0.021843,0.602478,0.841018,-0.099454,0.725952,0.930545,0.236801,0.943036,-0.005292,...,-0.041737,-0.042139,-0.037937,-0.052622,-0.068335,-0.040461,-0.039720,-0.061276,-0.041533,-0.040416
2,-0.596683,-0.244222,0.001055,0.149059,-0.449030,-0.153796,0.272994,-0.444606,0.203915,-0.174668,...,-0.041945,-0.041756,-0.038773,-0.053112,-0.063924,-0.039694,-0.039620,-0.062059,-0.041170,-0.040849
3,0.276354,-0.697907,-0.926980,-0.634224,-0.282329,-0.580624,-1.198670,0.136443,-1.134714,-0.036968,...,-0.039959,-0.036228,-0.038542,-0.052545,-0.068148,-0.040315,-0.039932,-0.061524,-0.042030,-0.039663
4,0.472700,-0.204151,0.059238,0.482899,0.144806,0.495885,0.669494,0.138797,0.369353,0.036591,...,-0.041444,-0.038877,-0.038015,-0.052744,-0.066103,-0.038627,-0.038831,-0.060161,-0.040898,-0.040289


In [16]:
from sklearn.svm import LinearSVC
model = LinearSVC(max_iter=10000)

scores = cross_val_score(model, df_normalized, y, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.5035461  0.4964539  0.5177305  0.4822695  0.51798561]
Accuracy: 0.50 (+/- 0.03)


In [23]:
from sklearn.svm import SVC
model = SVC(gamma='scale')

scores = cross_val_score(model, df_concat, y, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.5035461  0.5035461  0.5035461  0.5035461  0.50359712]
Accuracy: 0.50 (+/- 0.00)
